In [ ]:
# Libraries
import csv
import pandas as pd
import matplotlib.pyplot as plt

# Read in data
#Cloud_Sections
cs = pd.read_csv('CLOUD_SECTIONS_Cleaned.csv', delimiter = ',', encoding = 'latin1')
#Designed_Pipe_Center
desP = pd.read_csv('DESIGNED_Pipe_Centers_Cleaned.csv', delimiter = ',', encoding = 'latin1')
#Planes_Local_Global_Cleaned
p_info = pd.read_csv('Planes_Local_Global_Cleaned.csv', delimiter = ',', encoding = 'latin1')

In [ ]:
SectionedList = []
XsecList = []
YsecList = []
for s in range(cs.Section.nunique()): #creates a List of dataframes where each dataframe is a section
    SectionedList.append(cs[cs["Section"] == "Section-{" + str(s) + "}"])

for s in SectionedList: #for Section in SectionList, s is a dataframe for a section
    x = s.X
    y = s.Y
    XsecList.append(list(x))
    YsecList.append(list(y))
    #print(s)
    #plt.hist(s.X, bins=8, color = "orange")
    #print("Histogram of x values")
    #plt.show()
    #print(s.X)
   
    #plt.scatter(x, y)
    #plt.show()

In [ ]:
#create list showing the pipe centers
CenterList = []
for n in range(desP.Section.nunique()): #creates a List of dataframes where each dataframe is a section
    CenterList.append(desP[desP["Section"] == "Section-{" + str(n) + "}"])

print(CenterList[1])

In [ ]:
TuplePairsBySection = []
for i, j in zip(XsecList, YsecList):
    TuplePairsBySection.append(list(zip(i, j)))
count = 0
for C in TuplePairsBySection:
    plt.title("Section " + str(count))
    plt.xlim(0, max(x[0] for x in C) * 1.1)
    plt.ylim(0, max(y[1] for y in C) * 1.1)
    plt.scatter(*zip(*C))
    plt.scatter(CenterList[count].X, CenterList[count].Y, color = "red")
    a = plt.Circle((CenterList[count].iloc[0,1], CenterList[count].iloc[0,2]), radius = CenterList[count].iloc[0,4] * 1.05, color = "red")
    plt.gca().add_artist(a)
    b = plt.Circle((CenterList[count].iloc[1,1], CenterList[count].iloc[1,2]), radius = CenterList[count].iloc[1,4] * 1.05, color = "red")
    plt.gca().add_artist(b)
    plt.show()
    count += 1

In [ ]:
#OLD VERSION
count = 0
for i, j in zip(XsecList, YsecList):   
    plt.title("Section " + str(count))
    plt.xlim(0, max(i) + max(i) * 0.1)
    plt.ylim(0, max(j) + max(j) * 0.1)
    plt.scatter(i, j)
    plt.scatter(CenterList[count].X, CenterList[count].Y, color = "red")
    for n in range(len(CenterList[count])):
        #print(CenterList[count].iloc[n,])
        plt.Circle((CenterList[count].iloc[n,1], CenterList[count].iloc[n,2]), CenterList[count].iloc[n,4])
        plt.show()
    plt.show()
    count += 1
print("done")

In [33]:
for C in TuplePairsBySection:
    print(C)
    print(" ")

[(14.65, 44.56), (14.17, 43.33), (61.61, 59.48), (14.45, 44.45), (14.92, 45.03), (14.37, 43.62), (14.05, 42.2), (14.92, 44.96), (14.37, 43.97), (58.5, 69.35), (13.98, 42.39), (12.4, 40.77), (60.87, 60.03), (14.13, 47.14), (58.78, 62.92), (61.97, 59.21), (14.05, 42.95), (58.19, 64.98), (14.17, 43.22), (14.92, 44.84), (14.92, 45.24), (14.92, 45.11), (13.98, 42.55), (13.5, 41.53), (61.14, 59.77), (59.25, 70.78), (13.9, 42.47), (13.78, 41.8), (14.29, 43.5), (14.33, 46.81), (11.42, 40.34), (14.17, 43.14), (59.88, 60.98), (13.7, 41.73), (60.94, 59.96), (68.11, 58.33), (62.83, 58.74), (63.58, 58.41), (63.94, 58.34), (62.24, 59.01), (63.39, 58.53), (64.37, 58.26)]
 
[(59.65, 60.58), (60.51, 59.76), (9.68, 54.14), (9.53, 54.06), (60.04, 60.19), (11.97, 55.21), (10.0, 54.3), (10.59, 54.62), (59.37, 60.94), (11.97, 55.06), (10.31, 54.45), (11.42, 54.78), (61.02, 59.36), (58.98, 61.49), (8.98, 53.67), (60.12, 60.1), (11.77, 54.93), (61.69, 58.88), (61.85, 58.8), (60.2, 60.03), (11.81, 55.53), (10.